<a href="https://colab.research.google.com/github/alefram/blog-notebooks/blob/main/cuda_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA setup for experiment in C++ and Python



> ## C++

In [14]:
!pip install nvcc4jupyter

## Add plugin

In [15]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [18]:
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
    fflush(stdout); // Add this line to flush the output buffer
}

## Python

In [17]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 11.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1.1-cp311-cp311-linux_x86_64.whl size=660712 sha256=23e54ce22d2cdf69627d0a6ec6facdb8c3a2b79ca8fcdb16efb0b909d6adebac
  Stored in directory: /root/.cache/pip/wheels/49/0a/64/6530a5fde64f984ebb4992e38744fdfd2a61f510377b3a24d9
Successfully built pycuda


In [19]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda.compiler import SourceModule

# Kernel de CUDA escrito como una cadena de texto
mod = SourceModule("""
    __global__ void add_vectors(float *dest, float *a, float *b)
    {
        const int i = threadIdx.x + blockIdx.x * blockDim.x;
        dest[i] = a[i] + b[i];
    }
""")

# Obtener la función del kernel
add_vectors = mod.get_function("add_vectors")

# Crear arrays en la CPU
a = np.random.randn(4096).astype(np.float32)
b = np.random.randn(4096).astype(np.float32)
dest = np.zeros_like(a)

# Lanzar el kernel en la GPU
add_vectors(
    drv.Out(dest), drv.In(a), drv.In(b),
    block=(256, 1, 1),
    grid=(16, 1)
)

# Comprobar el resultado
print(f"Resultado en CPU (primer elemento): {a[0] + b[0]:.6f}")
print(f"Resultado en GPU (primer elemento): {dest[0]:.6f}")

Resultado en CPU (primer elemento): 0.418397
Resultado en GPU (primer elemento): 0.418397
